#DEEP LEARNING
##Lab 5

In [418]:
#Import Liabries
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots

In [419]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [420]:
#Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Deep Learning/AirPassengers.csv')
data.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [421]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Month        144 non-null    object
 1   #Passengers  144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [422]:
# Preprocessing
data['Month'] = pd.to_datetime(data['Month'])
data.set_index('Month', inplace=True)

# Handle missing values
data.fillna(method='ffill', inplace=True)
data

,#Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121
...,...
1960-08-01,606
1960-09-01,508
1960-10-01,461


In [423]:
# Visualize the time series
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['#Passengers'], mode='lines', name='Passengers'))
fig.update_layout(title='Airplane Passengers Time Series',
                  xaxis_title='Year',
                  yaxis_title='Passengers')
fig.show()


In [424]:
# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['#Passengers']])

In [425]:
# Decompose the time series
result = seasonal_decompose(data['#Passengers'], model='multiplicative', period=12)
trend = result.trend
seasonal = result.seasonal
residual = result.resid


In [426]:
# Plot the decomposed components
# Trend Plot
trend_fig = px.line(x=data.index, y=trend, title='Trend Component')
trend_fig.update_xaxes(title_text='Year')
trend_fig.update_yaxes(title_text='Trend')
trend_fig.show()

In [427]:
# Seasonal Plot
seasonal_fig = px.line(x=data.index, y=seasonal, title='Seasonal Component')
seasonal_fig.update_xaxes(title_text='Year')
seasonal_fig.update_yaxes(title_text='Seasonal')
seasonal_fig.show()

In [428]:
# Residual Plot
residual_fig = px.line(x=data.index, y=residual, title='Residual Component')
residual_fig.update_xaxes(title_text='Year')
residual_fig.update_yaxes(title_text='Residual')
residual_fig.show()

In [429]:
# Function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)


In [430]:
# Split the data in the ratio of 80:10:10
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
train_size = int(len(scaled_data) * train_ratio)
val_size = int(len(scaled_data) * val_ratio)
test_size = len(scaled_data) - train_size - val_size

train_data = scaled_data[:train_size]
val_data = scaled_data[train_size:train_size+val_size]
test_data = scaled_data[train_size+val_size:]

# Create sequences
seq_length = 12
X_train, y_train = create_sequences(train_data, seq_length)
X_val, y_val = create_sequences(val_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Convert data to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_val_tensor = torch.Tensor(X_val)
y_val_tensor = torch.Tensor(y_val)
X_test_tensor = torch.Tensor(X_test)
y_test_tensor = torch.Tensor(y_test)


In [431]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [432]:
# Define hyperparameters
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1
learning_rate = 0.1
num_epochs = 100
batch_size = 32

In [433]:
# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [434]:
train_losses = []
val_losses = []

# Training the model
best_val_loss = np.inf
best_model_state = None
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        if inputs.dim() == 3:
            inputs = inputs.squeeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            if inputs.dim() == 3:
                inputs = inputs.squeeze(1)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
    else:
        print("Validation loss stopped decreasing. Stopping training.")
        break


Epoch [1/100], Train Loss: 4.3443, Val Loss: 1.4748
Epoch [2/100], Train Loss: 1.8538, Val Loss: 1.0478
Epoch [3/100], Train Loss: 0.1783, Val Loss: 0.0917
Epoch [4/100], Train Loss: 0.0457, Val Loss: 0.2988
Validation loss stopped decreasing. Stopping training.


In [435]:
# Evaluate
model.load_state_dict(best_model_state)
model.eval()
test_loss = 0.0
predictions = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        if inputs.dim() == 3:
            inputs = inputs.squeeze(1)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
        predictions.extend(outputs.cpu().numpy().flatten())

test_loss /= len(test_loader.dataset)

y_true = scaler.inverse_transform(y_test)
y_pred = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f'Test Loss: {test_loss:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Test Loss: 0.1458
Mean Squared Error (MSE): 39128.9458
Root Mean Squared Error (RMSE): 197.8104
Mean Absolute Percentage Error (MAPE): 45.48%


In [436]:
# Plot training and validation loss
epochs = np.arange(1, num_epochs+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=epochs, y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Training and Validation Loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()


Performance of the model was not much affected by lower learning rate and sequence length, tried training and evaluating model on different Learning Rate values (0.0001, 0.001, 0.01, 0.1) and different sequence length values (2, 8, 12).